In [5]:
file_name = "backtest_1.ipynb" 
if file_name is None:
    file_name = __file__

In [6]:
# %%

import importlib

# ENVIRONMENT

import pandas as pd
import dotenv
import os

dotenv.load_dotenv(".env")

# DATA_PATH = os.environ["DATA_PATH"]
CATALOG_PATH = os.path.join(os.getcwd(), os.environ["CATALOG_PATH"])

# nautilus_trader imports

from nautilus_trader.model.position import Position
from nautilus_trader.model.objects import Price
from nautilus_trader.model.identifiers import Venue, InstrumentId, Symbol
from nautilus_trader.model.data import Bar, BarType, QuoteTick
from nautilus_trader.config import (
    BacktestVenueConfig,
    BacktestDataConfig,
    BacktestRunConfig,
    BacktestEngineConfig,
    RiskEngineConfig,
)
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.engine import BacktestResult
from nautilus_trader.trading.strategy import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.core.datetime import (
    dt_to_unix_nanos,
    maybe_unix_nanos_to_dt,
    unix_nanos_to_dt,
)
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.cache.cache import Cache
import decimal

# other imports
from pandas import Timestamp
import importlib
import matplotlib.pyplot as plt

# my packages
from put101 import indicators
import strategies
from strategies import bollinger_cluster
from put101 import utils
from put101 import vizz

# ---------------- CONFIGURATION ----------------
catalog = ParquetDataCatalog(CATALOG_PATH)
start = dt_to_unix_nanos(pd.Timestamp("2023-01-01 00:00:00"))

end = start + pd.Timedelta(days=30).value

venue_str = "SIM_EIGHTCAP"
venue = Venue(venue_str)
symbol_str = "EURUSD"
symbol = Symbol(symbol_str)
instrument_id_str = f"EURUSD.{venue}"

instrument_id = InstrumentId(symbol, venue)
script_name = os.path.basename(file_name).split(".")[0]


In [7]:
# %%
venue_configs = [
    BacktestVenueConfig(
        name=venue_str,
        oms_type="HEDGING",
        account_type="MARGIN",
        default_leverage=30,
        base_currency="USD",
        starting_balances=["10_000 USD"],
    ),
]

data_configs = [
    BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_cls=QuoteTick,
        instrument_id=instrument_id,
        start_time=start,
        end_time=end,
    ),
]

strategies = [
    ImportableStrategyConfig(
        strategy_path="strategies.base_1:PUT101Strategy",
        config_path="strategies.base_1:PUT101StrategyConfig",
        config=dict(
            instrument_id=instrument_id.value,
            bar_type=f"{instrument_id}-15-MINUTE-BID-INTERNAL",
        ),
    ),
]

configs = [
    BacktestRunConfig(
        engine=BacktestEngineConfig(
            strategies=strategies,
            trader_id="BACKTESTER-001",
            logging=LoggingConfig(
                log_level="INFO",
                log_level_file="DEBUG",
                log_directory="logs",
                log_file_name=f"backtest_{Timestamp.now()}.log",
                log_component_levels={"OrderMatchingEngine(SIM_EIGHTCAP)": "INFO"},
            ),
            risk_engine=RiskEngineConfig(
                bypass=True,  # Example of bypassing pre-trade risk checks for backtests
            ),
        ),
        data=data_configs,
        venues=venue_configs,
    )
]

node = BacktestNode(configs)



In [8]:
# %%
results = node.run()

2024-07-02T18:00:39.348016835Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2024-07-02T18:00:39.348085405Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2024-07-02T18:00:39.348094055Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2024-07-02T18:00:39.348100855Z [INFO] BACKTESTER-001.BacktestEngine: Copyright (C) 2015-2024. All rights reserved.
2024-07-02T18:00:39.348108115Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2024-07-02T18:00:39.348115435Z [INFO] BACKTESTER-001.BacktestEngine: 
2024-07-02T18:00:39.348122635Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2024-07-02T18:00:39.348129735Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2024-07-02T18:00:39.348136855Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀⠀
2024-07-02T18:00:

In [10]:
# %%
res = results[0]
backtest_start = maybe_unix_nanos_to_dt(res.backtest_start)
backtest_end = maybe_unix_nanos_to_dt(res.backtest_end)
res


BacktestResult(trader_id='BACKTESTER-001', machine_id='desktop', run_config_id='086e84a9a86eba681f0bda2549627354129b07e4599a4ba0e35393fb41dc882a', instance_id='3e70e51b-fc5d-45ac-b9d0-9bebd83e6bc6', run_id='944a5b29-527d-4206-a0c3-c34288687510', run_started=1719943240639408000, run_finished=1719943250803012000, backtest_start=1672704000507000000, backtest_end=1675123199503000000, elapsed_time=2419198.996, iterations=0, total_events=0, total_orders=0, total_positions=0, stats_pnls={'USD': {'PnL (total)': 0.0, 'PnL% (total)': 0.0, 'Max Winner': 0.0, 'Avg Winner': 0.0, 'Min Winner': 0.0, 'Min Loser': 0.0, 'Avg Loser': 0.0, 'Max Loser': 0.0, 'Expectancy': 0.0, 'Win Rate': 0.0}}, stats_returns={'Returns Volatility (252 days)': nan, 'Average (Return)': nan, 'Average Loss (Return)': nan, 'Average Win (Return)': nan, 'Sharpe Ratio (252 days)': nan, 'Sortino Ratio (252 days)': nan, 'Profit Factor': nan, 'Risk Return Ratio': nan})

In [2]:


engine = node.get_engine(res.run_config_id)
strategy: bollinger_cluster.BollingerCluster = 
cache: Cache = strategy.cache

main_t, main_s = strategy.get_main_plottable_indicators()
extra_plots = strategy.get_extra_plots()

layout = utils.get_layout(
    res=res,
    script_name=script_name,
    bars=strategy.bars,
    overlay_indicators=main_t,
    overlay_indicator_styles=main_s,
    extra_plots=extra_plots,
    positions=strategy.cache.positions(),
)

vizz.reset_output()
vizz.show(layout)


SyntaxError: invalid syntax (2622261504.py, line 9)